In [1]:
import numpy as np
import bioinfo4 as bio

# Ultrametric Evolutionary Trees

 ```R
UPGMA(D, n) 
       Clusters ← n single-element clusters labeled 1, ... , n
       construct a graph T with n isolated nodes labeled by single elements 1, ... , n
    for every node v in T 
        Age(v) ← 0
    while there is more than one cluster
        find the two closest clusters Ci and Cj  
        merge Ci and Cj into a new cluster Cnew with |Ci| + |Cj| elements
        add a new node labeled by cluster Cnew to T
        connect node Cnew to Ci and Cj by directed edges  
        Age(Cnew) ← DCi, Cj / 2
        remove the rows and columns of D corresponding to Ci and Cj 
        remove Ci and Cj from Clusters 
        add a row/column to D for Cnew by computing D(Cnew, C) for each C in Clusters
        add Cnew to Clusters
    root ← the node in T corresponding to the remaining cluster
    for each edge (v, w) in T
        length of (v, w) ← Age(v) - Age(w)
    return T
```

In [2]:
def input_to_D_array(ip):
    ip = ip.strip()
    ip = ip.split('\n')
    n = int(ip.pop(0))
    D_array = np.zeros((n,n))
    try:
        for i,row in enumerate(ip):
            D_array[i] = [int(k) for k in row.strip().split(" ")]
    except:
        for i,row in enumerate(ip):
            D_array[i] = [int(k) for k in row.strip().split("\t")]
    return D_array

def D_array_to_dict(D_array):
    n = D_array.shape[0]
    D_dict = {}
    for i in range(n-1):
        for k in range(i+1, n):
            D_dict[f"{i}-{k}"] = D_array[i,k]
    return D_dict

def input_to_D_dict(ip):
    D_array = input_to_D_array(ip)
    return D_array_to_dict(D_array)

def find_closest_clusters(D_dict):
    return min(D_dict.keys(), key=(lambda k: D_dict[k]))

In [13]:
def UPGMA(D_array,n,nn_count =None):
    D_dict = D_array_to_dict(D_array)

    Clusters = {}
    for i in range(n):
        Clusters[str(i)] = [str(i)]
    Clusters_ref = Clusters.copy()

    T = {}

    Age = {}
    for node in Clusters:
        Age[node] = 0
    if nn_count == None:
        nn_count = 1
    while len(Clusters) > 2 :

        # find closest clusters
        closest_clusters = find_closest_clusters(D_dict)
        cluster_1, cluster_2 = closest_clusters.split("-")

        if closest_clusters in D_dict:
            closest_distance = D_dict[closest_clusters]
        #update Distance and Clusters
        del D_dict[f"{cluster_1}-{cluster_2}"], Clusters[cluster_1], Clusters[cluster_2]

        #new_node = f"{cluster_1},{cluster_2}"
        new_node = str(n - 1 + nn_count)
        nn_count += 1
        Clusters_ref[new_node] = Clusters_ref[cluster_1] + Clusters_ref[cluster_2]    
        #Merge 2 cluster and update distance
        for i in Clusters:
            if f"{i}-{cluster_1}" in D_dict:
                Di_to_cluster1 = D_dict[f"{i}-{cluster_1}"].copy()
                del D_dict[f"{i}-{cluster_1}"]
            else:
                Di_to_cluster1 = D_dict[f"{cluster_1}-{i}"]
                del D_dict[f"{cluster_1}-{i}"]

            if f"{i}-{cluster_2}" in D_dict:
                Di_to_cluster2 = D_dict[f"{i}-{cluster_2}"].copy()
                del D_dict[f"{i}-{cluster_2}"]
            else:
                Di_to_cluster2 = D_dict[f"{cluster_2}-{i}"]
                del D_dict[f"{cluster_2}-{i}"]

            Di_new_node = (
                            (Di_to_cluster1*len(Clusters_ref[cluster_1]))+
                           (Di_to_cluster2*len(Clusters_ref[cluster_2]))
                           )/(len(Clusters_ref[cluster_1])+len(Clusters_ref[cluster_2]))    

            D_dict[f"{i}-{new_node}"] = Di_new_node

        Clusters[new_node] = Clusters_ref[cluster_1] + Clusters_ref[cluster_2]

        # update age, add new node to T and connect new node 
        Age[new_node] = closest_distance/2
        new_edge = f"{new_node}->{cluster_1}"
        edge_length = Age[new_node]- Age[cluster_1]
        T = bio.AddEdge2T(T,new_edge,edge_length)

        new_edge = f"{new_node}->{cluster_2}"
        edge_length = Age[new_node]- Age[cluster_2]
        T = bio.AddEdge2T(T,new_edge,edge_length)



    root =str(n - 1 + nn_count)
    closest_clusters = find_closest_clusters(D_dict)
    closest_distance = D_dict[closest_clusters]
    cluster_1, cluster_2 = closest_clusters.split("-")

    Age[root] = D_dict[find_closest_clusters(D_dict)]/2
    new_edge = f"{root}->{cluster_1}"
    edge_length = Age[root]- Age[cluster_1]
    T = bio.AddEdge2T(T,new_edge,edge_length)

    new_edge = f"{root}->{cluster_2}"
    edge_length = Age[root]- Age[cluster_2]
    T = bio.AddEdge2T(T,new_edge,edge_length)
    T = {k:v for k,v in sorted(T.items(),key=(lambda item: int(item[0].split("->")[0])))}

    return T

In [4]:
ip = """
4
0	20	17	11
20	0	20	13
17	20	0	10
11	13	10	0
"""

In [5]:
ip = """
4
0	3	4	3
3	0	4	5
4	4	0	2
3	5	2	0
"""

In [16]:
ip = """25
0 1155 892 1144 1046 1214 1181 703 639 788 665 786 940 883 1078 707 1030 970 911 1239 922 937 721 1100 1025 
1155 0 738 819 1210 796 971 678 732 964 842 1130 952 835 849 675 906 780 1242 1047 966 1176 1005 776 1227 
892 738 0 1077 1022 1152 1054 1125 653 853 928 839 1038 764 1240 698 833 920 758 1188 1183 628 656 1049 1001 
1144 819 1077 0 773 1236 1200 1201 877 1232 837 692 714 825 1220 856 1026 684 1031 769 863 1233 948 661 1219 
1046 1210 1022 773 0 672 1079 919 747 1112 821 737 834 915 1225 874 1013 1165 1122 1185 787 1129 633 809 676 
1214 796 1152 1236 672 0 710 647 1034 1098 762 706 1153 1115 1162 968 1218 890 770 731 955 783 727 696 960 
1181 971 1054 1200 1079 710 0 999 663 926 1059 889 992 687 1204 993 1243 830 913 996 1149 813 1000 748 1029 
703 678 1125 1201 919 647 999 0 963 767 899 983 866 683 1139 1163 841 930 945 708 794 1084 984 977 862 
639 732 653 877 747 1034 663 963 0 752 1123 728 645 702 1118 1119 635 888 798 740 1178 927 891 725 1222 
788 964 853 1232 1112 1098 926 767 752 0 682 935 781 1148 1191 943 1072 907 1135 1158 921 1027 1173 859 818 
665 842 928 837 821 762 1059 899 1123 682 0 1117 1126 857 1069 829 1150 1089 858 990 938 771 1087 695 1172 
786 1130 839 692 737 706 889 983 728 935 1117 0 827 627 685 730 870 716 1132 1168 933 824 697 1110 1091 
940 952 1038 714 834 1153 992 866 645 781 1126 827 0 987 1171 650 944 1199 768 1075 801 670 679 1186 744 
883 835 764 825 915 1115 687 683 702 1148 857 627 987 0 763 1108 912 746 1245 814 881 979 1189 1143 951 
1078 849 1240 1220 1225 1162 1204 1139 1118 1191 1069 685 1171 763 0 657 718 667 981 1208 803 1023 1074 973 1113 
707 675 698 856 874 968 993 1163 1119 943 829 730 650 1108 657 0 799 1160 848 882 931 832 668 1086 1145 
1030 906 833 1026 1013 1218 1243 841 635 1072 1150 870 944 912 718 799 0 1213 646 1182 1099 760 815 1048 861 
970 780 920 684 1165 890 830 930 888 907 1089 716 1199 746 667 1160 1213 0 1217 991 1202 1231 1012 1114 949 
911 1242 758 1031 1122 770 913 945 798 1135 858 1132 768 1245 981 848 646 1217 0 843 711 886 790 959 851 
1239 1047 1188 769 1185 731 996 708 740 1158 990 1168 1075 814 1208 882 1182 991 843 0 1039 997 1095 1247 789 
922 966 1183 863 787 955 1149 794 1178 921 938 933 801 881 803 931 1099 1202 711 1039 0 1103 680 1042 1063 
937 1176 628 1233 1129 783 813 1084 927 1027 771 824 670 979 1023 832 760 1231 886 997 1103 0 629 962 1166 
721 1005 656 948 633 727 1000 984 891 1173 1087 697 679 1189 1074 668 815 1012 790 1095 680 629 0 995 1134 
1100 776 1049 661 809 696 748 977 725 859 695 1110 1186 1143 973 1086 1048 1114 959 1247 1042 962 995 0 823 
1025 1227 1001 1219 676 960 1029 862 1222 818 1172 1091 744 951 1113 1145 861 949 851 789 1063 1166 1134 823 0 
"""

In [21]:
D_array = input_to_D_array(ip)
n = D_array.shape[0]
T= UPGMA(D_array,n)

In [19]:
for edge, length in T.items():
    print(f"{edge}:{length:.3f}")

0->32:332.500
1->40:398.750
2->26:314.000
3->31:330.500
4->27:316.500
5->29:323.500
6->44:451.250
7->29:323.500
8->28:317.500
9->37:367.500
10->32:332.500
11->25:313.500
12->30:325.000
13->25:313.500
14->33:333.500
15->30:325.000
16->28:317.500
17->33:333.500
18->34:355.500
19->35:359.750
20->34:355.500
21->26:314.000
22->27:316.500
23->31:330.500
24->42:435.167
25->11:313.500
25->13:313.500
25->36:50.250
26->2:314.000
26->21:314.000
26->39:82.625
27->4:316.500
27->22:316.500
27->38:65.375
28->8:317.500
28->16:317.500
28->39:79.125
29->5:323.500
29->7:323.500
29->35:36.250
30->12:325.000
30->15:325.000
30->38:56.875
31->3:330.500
31->23:330.500
31->40:68.250
32->0:332.500
32->10:332.500
32->37:35.000
33->14:333.500
33->17:333.500
33->36:30.250
34->18:355.500
34->20:355.500
34->41:64.938
35->19:359.750
35->29:36.250
35->42:75.417
36->25:50.250
36->33:30.250
36->44:87.500
37->9:367.500
37->32:35.000
37->45:101.583
38->27:65.375
38->30:56.875
38->41:38.562
39->26:82.625
39->28:79.125
39->

In [9]:
ip = """	Cow	Pig	Horse	Mouse	Dog	Cat	Turkey	Civet	Human
Cow	0	295	300	524	1077	1080	978	941	940
Pig	295	0	314	487	1071	1088	1010	963	966
Horse	300	314	0	472	1085	1088	1025	965	956
Mouse	524	487	472	0	1101	1099	1021	962	965
Dog	1076	1070	1085	1101	0	818	1053	1057	1054
Cat	1082	1088	1088	1098	818	0	1070	1085	1080
Turkey	976	1011	1025	1021	1053	1070	0	963	961
Civet	941	963	965	962	1057	1085	963	0	16
Human	940	966	956	965	1054	1080	961	16	0"""
ip = ip.split("\n")
species = ip.pop(0).strip().split("\t")
n = len(species)
D = np.zeros((n,n))
for row,value in enumerate(ip):
    D[row] = value.split("\t")[1:]

In [10]:
n = D.shape[0]
UPGMA(D, n)

{'0->10': 147.5,
 '1->10': 147.5,
 '2->11': 153.5,
 '3->12': 247.16666666666666,
 '4->13': 409.0,
 '5->13': 409.0,
 '6->15': 496.5,
 '7->9': 8.0,
 '8->9': 8.0,
 '9->7': 8.0,
 '9->8': 8.0,
 '9->14': 470.625,
 '10->0': 147.5,
 '10->1': 147.5,
 '10->11': 6.0,
 '11->2': 153.5,
 '11->10': 6.0,
 '11->12': 93.66666666666666,
 '12->3': 247.16666666666666,
 '12->11': 93.66666666666666,
 '12->14': 231.45833333333334,
 '13->4': 409.0,
 '13->5': 409.0,
 '13->16': 129.8571428571429,
 '14->9': 470.625,
 '14->12': 231.45833333333334,
 '14->15': 17.875,
 '15->6': 496.5,
 '15->14': 17.875,
 '15->16': 42.35714285714289,
 '16->13': 129.8571428571429,
 '16->15': 42.35714285714289}

# Neighbor joining matrix

In [123]:
import copy


In [124]:
def MkEdge(node1,node2):
    return f"{node1}->{node2}"
def RvEdge(edge):
    node1,node2 = edge.split("->")
    return node2 + "->" + node1

In [215]:
def FindTotalDistance(D_dict):
    Total_distance = {}
    for key in D_dict:
        node1, node2 = key.split("-")
        if node1 in Total_distance :
            Total_distance[node1] = Total_distance[node1] + D_dict[key]
        else:
            Total_distance[node1] = D_dict[key]

        if node2 in Total_distance :
            Total_distance[node2] = Total_distance[node2] + D_dict[key]
        else:
            Total_distance[node2] = D_dict[key]
    return Total_distance

def FindN(D_dict):
    x = len(D_dict)
    count = 1
    while x > 0:
        x -= count
        count += 1
    return count

def NJD(D_dict, Total_distance = None):
    if Total_distance == None:
        Total_distance = FindTotalDistance(D_dict)
    D_copy = D_dict.copy()
    n = FindN(D_dict)
    for key in D_copy:
        node1,node2 = key.split("-")
        new_distance = (n-2)*D_dict[key] - Total_distance[node1] - Total_distance[node2]
        D_copy[key] = new_distance
    return D_copy

def RemoveNode(D_dict,node):
    D_copy = copy.deepcopy(D_dict)
    keys = list(D_copy.keys())
    for key in keys:
        if node in key.split("-"):
            del D_copy[key]
    return D_copy

def FindDistance(D_dict,i,j):
    try:
        return D_dict[f"{i}-{j}"]
    except:
        return D_dict[f"{j}-{i}"]
    
def SortTree(T):
    T = {k:v for k,v in sorted(T.items(),key=(lambda item: item[0]))}
    T = {k:v for k,v in sorted(T.items(),key=(lambda item: int(item[0].split("->")[0])))}
    return T

```R
NeighborJoining(D)
    n ← number of rows in D
    if n = 2
        T ← tree consisting of a single edge of length D1,2
        return T
    D* ← neighbor-joining matrix constructed from the distance matrix D
    find elements i and j such that D*i,j is a minimum non-diagonal element of D*
    Δ ← (TotalDistanceD(i) - TotalDistanceD(j)) /(n - 2)
    limbLengthi ← (1/2)(Di,j + Δ)
    limbLengthj ← (1/2)(Di,j - Δ)
    add a new row/column m to D so that Dk,m = Dm,k = (1/2)(Dk,i + Dk,j - Di,j) for any k
    D ← D with rows i and j removed
    D ← D with columns i and j removed
    T ← NeighborJoining(D)
    add two new limbs (connecting node m with leaves i and j) to the tree T
    assign length limbLengthi to Limb(i)
    assign length limbLengthj to Limb(j)
    return T
```

In [253]:
def NeighborJoining(D_dict, rc_count=None):
    D_dict = copy.deepcopy(D_dict)
    ###recursive count and new node count 

    if rc_count == None:
        rc_count = 0
        global nn_count
        nn_count = 0

    else:
        rc_count +=1
    n = FindN(D_dict)
    #base
    if len(D_dict) == 1:
        T = {}
        #Make 2-node Tree
        node1, node2 = list(D_dict.keys())[0].split("-")
        edge = MkEdge(node1,node2)
        length = list(D_dict.values())[0]
        length = round(length,3)
        rv_edge = RvEdge(edge)
        T[edge] = length
        T[rv_edge] = length
        return T
    Total_distance = FindTotalDistance(D_dict)
    NJD_dict = NJD(D_dict, Total_distance)
    neighbors_ij = min(NJD_dict.keys(),key=(lambda k: NJD_dict[k]))
    node_i, node_j = neighbors_ij.split("-")
    delta_ij = (Total_distance[node_i] - Total_distance[node_j])/(n-2)
    limbLength_i = round((D_dict[neighbors_ij] + delta_ij)/2,3)
    limbLength_j = round((D_dict[neighbors_ij] - delta_ij)/2,3)
    #
    new_node = str(n + rc_count + nn_count)
    nn_count += 1
    for node_k in Total_distance:
        if node_k != node_i and node_k != node_j:
            D_i_k = FindDistance(D_dict, node_i, node_k)
            D_j_k = FindDistance(D_dict, node_j, node_k)
            D_nn = D_dict[neighbors_ij]
            D_k_nn = (D_i_k + D_j_k - D_nn)/2
            D_dict[f"{node_k}-{new_node}"] = D_k_nn
    #
    D_dict = RemoveNode(D_dict,node_i)
    D_dict = RemoveNode(D_dict,node_j)
    
    T = NeighborJoining(D_dict,rc_count)
    
    #
    edge1 = MkEdge(new_node, node_i)
    rv_edge1 = RvEdge(edge1)
    T[edge1] = limbLength_i
    T[rv_edge1] = limbLength_i

    edge2 = MkEdge(new_node, node_j)
    rv_edge2 = RvEdge(edge2)
    T[edge2] = limbLength_j
    T[rv_edge2] = limbLength_j
    
    return T

In [130]:
ip = """4
0	23	27	20
23	0	30	28
27	30	0	30
20	28	30	0"""

In [261]:
ip = """4
0	14	17	17
14	0	7	13
17	7	0	16
17	13	16	0"""

In [254]:
ip = """32
0 1346 1088 1183 1139 1029 1299 1727 1872 1659 1076 1459 1597 1381 1785 1893 1215 1047 1883 1925 1975 1579 1249 1604 1283 1090 1643 1349 1501 1782 1226 1165 
1346 0 1617 1266 1757 1125 1791 1557 1718 1531 2004 1769 1884 1429 1832 1564 1980 1191 1690 1377 1724 2001 1658 1288 1529 1169 1498 1310 1058 2007 1327 1685 
1088 1617 0 1795 1260 1809 1261 1565 1485 1732 1424 1281 1997 1687 1571 1694 2033 2043 1974 1958 1607 1608 1987 1385 1096 1834 2036 1225 1520 1180 1850 1983 
1183 1266 1795 0 1790 1567 1301 1621 1965 1671 1765 1290 2044 1080 1566 1670 1950 1331 1855 1909 1171 1535 1306 1167 1544 1408 1845 1830 2041 1145 1210 1397 
1139 1757 1260 1790 0 1033 1113 1515 1093 1656 1781 1652 1361 1371 1612 1961 1523 1275 1111 1763 1149 1278 1602 1142 1363 1837 1864 1869 1248 1119 1365 1903 
1029 1125 1809 1567 1033 0 2002 1156 1518 1469 1572 1947 1839 1660 1537 1835 2000 1416 1849 1091 1816 1298 1610 1317 1155 1292 1684 1316 1712 1359 1541 1716 
1299 1791 1261 1301 1113 2002 0 1273 1504 1434 1852 1595 1923 1649 1044 1928 1031 1293 1695 1726 1897 1176 1982 1190 1186 1582 1187 1630 1110 1748 1988 1971 
1727 1557 1565 1621 1515 1156 1273 0 1025 1420 1229 1525 1558 1475 1820 2027 1934 1158 1252 1742 1218 1770 1984 1362 1588 2015 1825 1948 1536 1136 1314 1409 
1872 1718 1485 1965 1093 1518 1504 1025 0 1143 1103 1973 1422 1262 1655 1412 1569 1039 1706 2008 1321 1560 1427 1154 1574 1799 1913 1483 1199 1401 1989 1502 
1659 1531 1732 1671 1656 1469 1434 1420 1143 0 1436 1703 1296 1396 1755 1036 1184 1917 1144 1966 1915 1419 1900 1738 1648 1624 1599 1400 1464 1207 1473 1486 
1076 2004 1424 1765 1781 1572 1852 1229 1103 1436 0 1304 1591 1797 1493 1120 1060 1390 1479 1451 1402 1085 1300 1905 1752 1027 1470 1448 1495 1589 1089 1697 
1459 1769 1281 1290 1652 1947 1595 1525 1973 1703 1304 0 1779 1303 1688 1421 1743 1637 1810 1073 1203 1749 1393 1269 1807 1492 1214 1192 1540 1064 1389 2040 
1597 1884 1997 2044 1361 1839 1923 1558 1422 1296 1591 1779 0 1116 1863 1568 1094 1940 1387 1600 1861 1995 1885 1152 1326 1879 1868 1750 1731 1461 1682 1737 
1381 1429 1687 1080 1371 1660 1649 1475 1262 1396 1797 1303 1116 0 1812 1661 1439 1423 1028 1761 2029 1270 1548 1336 1413 1372 1640 1904 2018 1841 1728 1228 
1785 1832 1571 1566 1612 1537 1044 1820 1655 1755 1493 1688 1863 1812 0 1482 1127 1798 1067 1946 1445 1471 2047 2031 1263 1689 1786 1179 1527 1901 1645 1833 
1893 1564 1694 1670 1961 1835 1928 2027 1412 1036 1120 1421 1568 1661 1482 0 1160 1130 1181 1098 1516 2039 1201 1960 1146 1350 1066 1311 1410 1392 1874 1631 
1215 1980 2033 1950 1523 2000 1031 1934 1569 1184 1060 1743 1094 1439 1127 1160 0 1744 1236 1677 1562 1291 1764 1821 1082 1264 1095 1070 1686 1848 1681 1700 
1047 1191 2043 1331 1275 1416 1293 1158 1039 1917 1390 1637 1940 1423 1798 1130 1744 0 1309 1609 1644 1808 1514 1453 1294 1622 1586 1195 1487 1991 1877 1642 
1883 1690 1974 1855 1111 1849 1695 1252 1706 1144 1479 1810 1387 1028 1067 1181 1236 1309 0 1175 1937 2020 1708 1663 1166 1886 1509 2009 1920 1041 1977 1438 
1925 1377 1958 1909 1763 1091 1726 1742 2008 1966 1451 1073 1600 1761 1946 1098 1677 1609 1175 0 2012 1641 1867 1710 1198 1351 1693 1842 1907 1996 1875 2003 
1975 1724 1607 1171 1149 1816 1897 1218 1321 1915 1402 1203 1861 2029 1445 1516 1562 1644 1937 2012 0 1132 1277 1050 1618 1030 1702 1889 1715 1234 1357 1592 
1579 2001 1608 1535 1278 1298 1176 1770 1560 1419 1085 1749 1995 1270 1471 2039 1291 1808 2020 1641 1132 0 1159 1212 1583 1871 1494 1053 1055 1335 2022 1623 
1249 1658 1987 1306 1602 1610 1982 1984 1427 1900 1300 1393 1885 1548 2047 1201 1764 1514 1708 1867 1277 1159 0 1729 1255 1450 1680 1284 1605 1906 1851 1632 
1604 1288 1385 1167 1142 1317 1190 1362 1154 1738 1905 1269 1152 1336 2031 1960 1821 1453 1663 1710 1050 1212 1729 0 1554 1853 1994 1129 1202 1508 1312 1657 
1283 1529 1096 1544 1363 1155 1186 1588 1574 1648 1752 1807 1326 1413 1263 1146 1082 1294 1166 1198 1618 1583 1255 1554 0 1100 1259 1407 1814 1754 1051 1107 
1090 1169 1834 1408 1837 1292 1582 2015 1799 1624 1027 1492 1879 1372 1689 1350 1264 1622 1886 1351 1030 1871 1450 1853 1100 0 1131 1801 1705 1394 1442 1322 
1643 1498 2036 1845 1864 1684 1187 1825 1913 1599 1470 1214 1868 1640 1786 1066 1095 1586 1509 1693 1702 1494 1680 1994 1259 1131 0 1308 1478 1629 1037 1140 
1349 1310 1225 1830 1869 1316 1630 1948 1483 1400 1448 1192 1750 1904 1179 1311 1070 1195 2009 1842 1889 1053 1284 1129 1407 1801 1308 0 1034 1620 1914 1083 
1501 1058 1520 2041 1248 1712 1110 1536 1199 1464 1495 1540 1731 2018 1527 1410 1686 1487 1920 1907 1715 1055 1605 1202 1814 1705 1478 1034 0 1217 1244 1811 
1782 2007 1180 1145 1119 1359 1748 1136 1401 1207 1589 1064 1461 1841 1901 1392 1848 1991 1041 1996 1234 1335 1906 1508 1754 1394 1629 1620 1217 0 1521 1692 
1226 1327 1850 1210 1365 1541 1988 1314 1989 1473 1089 1389 1682 1728 1645 1874 1681 1877 1977 1875 1357 2022 1851 1312 1051 1442 1037 1914 1244 1521 0 1162 
1165 1685 1983 1397 1903 1716 1971 1409 1502 1486 1697 2040 1737 1228 1833 1631 1700 1642 1438 2003 1592 1623 1632 1657 1107 1322 1140 1083 1811 1692 1162 0 
"""

In [262]:
D_array = input_to_D_array(ip)
D_dict = input_to_D_dict(ip)
D_dict

{'0-1': 14.0, '0-2': 17.0, '0-3': 17.0, '1-2': 7.0, '1-3': 13.0, '2-3': 16.0}

In [263]:
T = NeighborJoining(D_dict)
T = SortTree(T)
for edge, length in T.items():
    print(f"{edge}:{length:.3f}")

0->4:9.000
1->5:2.000
2->5:5.000
3->4:8.000
4->0:9.000
4->3:8.000
4->5:3.000
5->1:2.000
5->2:5.000
5->4:3.000
